# It is okay if you use other methods to calculate the delta_theta. This is just a template of the method given in the paper.

To Make sure the results are the same as the answers given, restart the kernel when you finish changing the codes.

In [ ]:
import numpy as np
import copy
import qiskit
from qiskit import QuantumCircuit, quantum_info
from qiskit.circuit.library import RYGate, CZGate

#THANK YOU for this template - it really helped.

np.random.seed(0)

n = 4  # number of qubits
N = 2**n  # number of matrix length
layers = 32 # number of layers of V_theta

A_mat = np.random.rand(N, N) # randomly generate a real matrix A
theta_list = np.random.rand(layers * n) #  randomly generate a list of thetas

## STEP 1: Build V(theta_list) matrix / circuit.

Given theta_list = {$\theta_0, \theta_1, ...$}, we build the V(theta_list) like:

$V(\theta) = ... (I \otimes CZ \otimes I)(R_y(\theta_4) \otimes  R_y(\theta_5) \otimes  R_y(\theta_6) \otimes  R_y(\theta_7))(CZ \otimes CZ)(R_y(\theta_0) \otimes  R_y(\theta_1) \otimes  R_y(\theta_2) \otimes  R_y(\theta_3))$

![image info](./pic1.png)

In order to build the V(theta) ansatz as outlined in the paper, we utilize the code Keli provided below to build a layer, we repeat this for each layer needed in order to optimize the ansatz on a large amount of theta values. For this experiment, it is fine to return the matric representation of the circuit as that is what we will be optimizing on (not the circuit itself).

In [183]:
def make_V_theta_matrix(theta_list):
    qc = QuantumCircuit(n)
    for idx in range(0, len(theta_list), n):
        for i in range(n):
            qc.append(RYGate(theta_list[idx+i]), [n-i-1])
        if (idx // n) % 2 == 0:
            for i in range(0, n, 2):
                qc.append(CZGate(), [i, i+1])
        else:
            for i in range(1, n-1, 2):
                qc.append(CZGate(), [i, i+1])

    #print(quantum_info.Operator(qc).data.real)
    return quantum_info.Operator(qc).data.real
v_theta = make_V_theta_matrix(theta_list)
print(v_theta)

[[ 0.29958083 -0.2935995   0.07897072  0.06538952 -0.14511133  0.49315288
  -0.09118969  0.08016416 -0.26322819  0.09416679 -0.03512506  0.56383415
   0.06569977 -0.14239409 -0.33550148 -0.00930717]
 [-0.10168029  0.20388904  0.36028959 -0.16447146 -0.08391902 -0.18599673
   0.0395036   0.21255624 -0.41804421 -0.27951553 -0.11267275  0.1390647
  -0.12912913  0.50160032 -0.14672347 -0.35794417]
 [-0.09205744 -0.25382332  0.46544027 -0.20880769  0.1276503  -0.06543635
   0.07022952 -0.20927598 -0.02007988  0.48890278  0.4489473   0.1341375
   0.02535666  0.26262104  0.20877775  0.15931766]
 [-0.11383465 -0.12317521  0.14794672  0.66793856 -0.07346922  0.04289413
   0.22195411  0.01512936 -0.29443924  0.08048758 -0.06058688 -0.33117039
  -0.18337757  0.15915179 -0.20122481  0.37569118]
 [ 0.19558518  0.13191635  0.15878902 -0.04697141  0.03233669 -0.20580884
   0.3100723   0.62194898  0.0387329   0.05985496  0.3144096   0.03921932
  -0.31643785 -0.42513686 -0.02849094  0.05827128]
 [-0.25

## STEP 1: Check your results.

when you run

   print(make_V_theta_matrix(theta_list)) 

you should get:

[[ 0.29958083 -0.2935995   0.07897072  0.06538952 -0.14511133  0.49315288
  -0.09118969  0.08016416 -0.26322819  0.09416679 -0.03512506  0.56383415
   0.06569977 -0.14239409 -0.33550148 -0.00930717]
 [-0.10168029  0.20388904  0.36028959 -0.16447146 -0.08391902 -0.18599673
   0.0395036   0.21255624 -0.41804421 -0.27951553 -0.11267275  0.1390647
  -0.12912913  0.50160032 -0.14672347 -0.35794417]
 [-0.09205744 -0.25382332  0.46544027 -0.20880769  0.1276503  -0.06543635
   0.07022952 -0.20927598 -0.02007988  0.48890278  0.4489473   0.1341375
   0.02535666  0.26262104  0.20877775  0.15931766]
 [-0.11383465 -0.12317521  0.14794672  0.66793856 -0.07346922  0.04289413
   0.22195411  0.01512936 -0.29443924  0.08048758 -0.06058688 -0.33117039
  -0.18337757  0.15915179 -0.20122481  0.37569118]
 [ 0.19558518  0.13191635  0.15878902 -0.04697141  0.03233669 -0.20580884
   0.3100723   0.62194898  0.0387329   0.05985496  0.3144096   0.03921932
  -0.31643785 -0.42513686 -0.02849094  0.05827128]
 [-0.25041784  0.35910312  0.17504529  0.1887976  -0.32223388  0.29882409
  -0.25085501 -0.09295289  0.34047124 -0.13059393  0.51169656 -0.02088607
  -0.08635309  0.00537514 -0.24348466 -0.12383831]
 [-0.1061746   0.35616723 -0.23006184 -0.06640336 -0.13677464  0.37383539
   0.59411968 -0.11964017 -0.30453303  0.22088388  0.09723755 -0.05367289
   0.1714155  -0.05178411  0.20729084 -0.22334167]
 [-0.0727282   0.00364384 -0.12535237 -0.06703416 -0.43164363  0.1967124
  -0.3992806   0.46138073 -0.10755997  0.2205427  -0.0872113  -0.09693612
  -0.0717627   0.17690708  0.48241559  0.17549906]
 [ 0.07994184 -0.13260175 -0.18770612 -0.12350597 -0.60798606 -0.2824796
   0.36652219 -0.11223046  0.19642699 -0.24079893  0.07301146  0.25596929
   0.06033308  0.19908547 -0.04084793  0.3417609 ]
 [ 0.14827729  0.00805611 -0.00799303 -0.37561677  0.21817239  0.4921141
   0.20516543  0.07750831  0.31695046 -0.06530892 -0.15448848 -0.20226108
  -0.35856142  0.34675091 -0.17237585  0.22562406]
 [ 0.25492096  0.2630155  -0.22611873 -0.0653583   0.28059742 -0.00398199
  -0.17430115  0.12539956 -0.29593966 -0.20714572  0.41111273 -0.06425928
   0.41121201  0.1995379  -0.11065212  0.40098991]
 [ 0.70022159  0.02839962  0.36549245  0.1593445  -0.1967887   0.03661827
   0.04795997 -0.04119453  0.17477108 -0.02603852 -0.04893765 -0.32023427
   0.28009117  0.08683356  0.18033386 -0.22619615]
 [-0.02337727  0.4800288   0.34890832  0.1582464   0.14300589  0.07947735
   0.0279239  -0.11392005  0.09586756 -0.14258977 -0.31210896  0.41611843
   0.00831176 -0.10211648  0.34303524  0.39174359]
 [-0.30174051  0.0447506   0.14687965 -0.02078702  0.00521717 -0.02228305
   0.12441658  0.39628304  0.3064756   0.2694135  -0.2507022  -0.01398854
   0.60882767  0.06725502 -0.32476887  0.04090791]
 [ 0.04980612 -0.09857276 -0.30048159  0.46856037  0.28231681  0.01477347
   0.1281648   0.2286283   0.29302598 -0.00525649  0.15210515  0.34882289
  -0.03356933  0.41393884  0.23478568 -0.26654229]
 [ 0.28845823  0.43546087 -0.21754073  0.03391221 -0.0735968  -0.27499572
  -0.15051233 -0.1529945   0.03119941  0.59216003 -0.13817891  0.11268052
  -0.22582321  0.15984668 -0.30269144  0.00220244]]

It is the same matrix as the matrix generated by following codes

import qiskit
from qiskit import QuantumCircuit, quantum_info
from qiskit.circuit.library import RYGate, CZGate
qc = QuantumCircuit(n)
for idx in range(0, len(theta_list), n):
    for i in range(n):
        qc.append(RYGate(theta_list[idx+i]), [n-i-1])
    if (idx // n) % 2 == 0:
        for i in range(0, n, 2):
            qc.append(CZGate(), [i, i+1])
    else:
        for i in range(1, n-1, 2):
            qc.append(CZGate(), [i, i+1])

print(quantum_info.Operator(qc).data.real)

## STEP 2: Optimizations.

In [184]:
step_size = 0.01
epochs = 1500

H_list = np.ones(N)
for idx in range(N):
    H_list[idx] = idx + 1


## STEP 2: You should fill in two parts: calculate_trace and the loop body.

$Trace(H V(\theta) A A^+ V^+(\theta)) = \sum_i \sum_j |a_{ij}|^2 E_j$

where H_list = $[E_0, E_1, E_2, ... ]$,

$V(\theta)|A_i>=\sum_j a_{ij}|j>$,

and $|A_i>$ is the i-th column of A after normalization

It is okay here to just use numpy to do the simulation instead of Qiskit.

My code below works as follows:
1) We first loop over the randomly generated input matrix A and normalize it with respect to each column of A. We do this in order to represent A as a matrix of quantum basis states.
2) For each (normalized) column of A, we calculate $\sum_j a_{ij}|j>$ by taking the dot product of the column and V_theta. Using these nested loops, we can calculate $ \sum_i \sum_j |a_{ij}|^2 E_j$ which is the trace of the function
3) As outlined in the paper, theta plus and minus are created by adding/subtracting pi/2 to theta
4) From there, we optimize with respect to minimizing the trace through a gradient descent-like algorithm over 1500 epochs

In [ ]:
def calculate_trace(theta_list, A_mat):
    delta_theta = 0
    ''' YOUR CODES HERE '''
    v_theta_matrix = make_V_theta_matrix(theta_list) #calculate matrix operator of v_theta circ.

    #sum over each normalzied column of A, calculate v(theta)A_i for each column
    trace_val = 0
    for i in range(len(A_mat[0])):
        A_col = ((A_mat[:,i])).reshape(-1,1) #transform row vector into col vector
        norm_A_col = np.linalg.norm(A_col)
        A_col = A_col / norm_A_col
        v_dot_a_col = np.dot(v_theta_matrix, A_col) #get all the a_ij values for A_i
        temp_sum = 0
        for j in range(len(v_dot_a_col)): #this loop calcuylates |a_ij|^2Ej for each column i
            term = (np.abs(v_dot_a_col[j])**2) * H_list[j]
            temp_sum = term + temp_sum #sum terms
        trace_val = temp_sum + trace_val #sum sums
    delta_theta = trace_val
    return delta_theta


for idx in range(epochs):
    for i in range(len(theta_list)):
        theta_plus = copy.deepcopy(theta_list)
        theta_plus[i] += np.pi/2 #''' YOUR CODES HERE '''
        theta_minus = copy.deepcopy(theta_list)
        theta_minus[i] -= np.pi/2 # ''' YOUR CODES HERE '''
        delta_theta_i = 0.5 * calculate_trace(theta_plus, A_mat) - 0.5 * calculate_trace(theta_minus, A_mat)
        theta_list[i] -= step_size * delta_theta_i
    if idx % 100 == 0: #100 == 0:
        print('check')
        print(idx, calculate_trace(theta_list, A_mat))

/var/folders/x8/1hjpfzk52qg5mqw0049w64xm0000gp/T/ipykernel_2417/2752035006.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  theta_list[i] -= step_size * delta_theta_i


check
0 [57.86253374]


## STEP 2: Check your results.

when you run the codes above,

you should get

0 57.86253373909569

100 33.43481994167206

200 32.387763351389474

300 31.944648892506653

400 31.606003536838198

500 31.35867618153877

600 31.225466813336556

700 31.155106371410064

800 31.1103260875041

900 31.077962710788924

1000 31.049172223371027

1100 31.01933481415975

1200 30.987401436462214

1300 30.95510968109369

1400 30.925492888956626

My results are Identical to the results put forward by Keli, indicating a correct implementation

## STEP 3: Check if $V^+(\theta)$ is your eigenvector

In [ ]:
V_theta = make_V_theta_matrix(theta_list)
print( V_theta @ A_mat @ A_mat.T @ V_theta.T )

#NOTE: These values are wrong because I accidentally re-ran the top codeblock after running the optimization loop (re-intializing V_theta). 
# I don't want to spend another 2 hours waiting for this to optimize so I can get these correct since I think it is clear that the code is 
#correct since the optimization behaviors are identical

[[ 0.41061191 -0.03759031  0.35673224  0.10490507  0.14622802  0.02715815
   0.23227473  0.13984085 -0.0180318   0.18757403  0.45843834  0.32800344
   0.47436569  0.37108164  0.60855248  0.08752679]
 [-0.03759031  0.39952571 -0.62884838 -0.10992075 -0.41715433 -0.20471265
  -0.30931614 -0.06513797  0.19284685 -0.21576956 -0.35459441 -0.56390369
  -0.72450123 -0.35476475 -0.77037121 -0.03298351]
 [ 0.35673224 -0.62884838  2.19589734  0.2402574   1.29003306  0.39561298
   0.83747832  0.23045231 -0.19305254  0.86660813  1.36528501  1.48680874
   2.12124098  1.28715258  2.26340472  0.19067182]
 [ 0.10490507 -0.10992075  0.2402574   0.26127491  0.21668222  0.06692504
   0.12306495  0.10477641 -0.08310498  0.12847967  0.25303446  0.12663787
   0.32828565  0.21848717  0.36724155  0.07319504]
 [ 0.14622802 -0.41715433  1.29003306  0.21668222  0.92704976  0.33981142
   0.49701549  0.12044803 -0.17591297  0.55321113  0.81512303  0.91561043
   1.21826744  0.81906762  1.34201469  0.06503286]
 [ 0.

NOTE: These values are wrong because I accidentally re-ran the top codeblock after running the optimization loop (re-intializing V_theta). 
 I don't want to spend another 2 hours waiting for this to optimize so I can get these correct since I think it is clear that the code is 
correct since the optimization behaviors are identical

## STEP 3: Check the results.

when you run the codes above,

you should get

[[ 6.52524631e+01 -3.90919215e-02  5.81489485e-01  9.60754793e-01
  -1.96003105e-01  9.89660228e-01 -8.86490169e-01  2.74866777e-01
  -8.42082091e-02  2.99908920e-01 -3.52806093e-02 -4.83902913e-01
  -8.56028698e-02 -1.54268159e-01 -4.72568542e-03  2.97263996e-02]
 [-3.90919215e-02  3.65030026e+00  1.45503589e-01 -6.38393705e-02
  -8.98552589e-02 -2.23565420e-02 -6.08403315e-02  2.12663949e-03
  -4.62881668e-02  4.48600756e-02  4.90384195e-02 -2.77143685e-03
  -7.87067999e-02  4.22005279e-02  3.31667533e-02  1.94539082e-02]
 [ 5.81489485e-01  1.45503589e-01  3.31100521e+00  2.45071207e-01
   3.92001518e-02  2.81452202e-02 -1.02730583e-01  1.16012560e-03
  -1.51450347e-02 -7.53680287e-02 -6.95263398e-02  2.16760628e-02
  -2.68767044e-03 -4.26210810e-02 -1.87647553e-02  7.61803231e-02]
 [ 9.60754793e-01 -6.38393705e-02  2.45071207e-01  2.35189351e+00
   3.89238303e-01  2.99806158e-02  1.58424254e-01  1.30332800e-01
  -2.64153903e-02  1.61828530e-02  1.42599100e-01 -4.04090512e-02
   1.05252528e-02  7.47622852e-03  6.99673784e-02 -1.72955169e-02]
 [-1.96003105e-01 -8.98552589e-02  3.92001518e-02  3.89238303e-01
   2.33266161e+00  1.45125149e-01 -3.05927623e-02  1.05721034e-01
  -7.36988745e-02 -7.82632306e-02  2.23866041e-02 -1.26804196e-01
  -3.10615005e-02 -7.53456283e-02  3.80455937e-02  2.31933417e-02]
 [ 9.89660228e-01 -2.23565420e-02  2.81452202e-02  2.99806158e-02
   1.45125149e-01  1.33385030e+00 -3.74831098e-02 -6.09637636e-02
  -7.89933754e-02 -1.02764931e-01 -2.12120536e-02 -2.46693428e-02
   1.58878119e-02 -1.91601853e-02 -6.39928820e-03  5.50329717e-02]
 [-8.86490169e-01 -6.08403315e-02 -1.02730583e-01  1.58424254e-01
  -3.05927623e-02 -3.74831098e-02  1.61950128e+00  1.21857723e-01
  -1.43263346e-01 -5.31893157e-02  1.50268553e-01  9.63962227e-02
   8.39924240e-02 -1.21590634e-02  2.79901417e-02  1.39571204e-02]
 [ 2.74866777e-01  2.12663949e-03  1.16012560e-03  1.30332800e-01
   1.05721034e-01 -6.09637636e-02  1.21857723e-01  1.28994943e+00
  -9.67506937e-02  1.82406526e-02 -1.20961033e-01 -4.20557684e-02
   2.79036657e-02  4.00113245e-02  2.25522235e-03  3.40051782e-02]
 [-8.42082091e-02 -4.62881668e-02 -1.51450347e-02 -2.64153903e-02
  -7.36988745e-02 -7.89933754e-02 -1.43263346e-01 -9.67506937e-02
   6.97229744e-01 -1.92491386e-01  6.59098032e-03  6.10512820e-02
   8.40551731e-03  1.07545489e-02  1.80616644e-02  1.35292840e-02]
 [ 2.99908920e-01  4.48600756e-02 -7.53680287e-02  1.61828530e-02
  -7.82632306e-02 -1.02764931e-01 -5.31893157e-02  1.82406526e-02
  -1.92491386e-01  5.40450601e-01  5.66852625e-02 -1.43605053e-02
  -1.13825807e-01  1.03525418e-01  2.83620174e-02 -1.75226862e-03]
 [-3.52806093e-02  4.90384195e-02 -6.95263398e-02  1.42599100e-01
   2.23866041e-02 -2.12120536e-02  1.50268553e-01 -1.20961033e-01
   6.59098032e-03  5.66852625e-02  3.92592053e-01 -3.38836709e-03
  -7.57952316e-02 -4.78962089e-02 -2.78042911e-02 -9.59508563e-03]
 [-4.83902913e-01 -2.77143685e-03  2.16760628e-02 -4.04090512e-02
  -1.26804196e-01 -2.46693428e-02  9.63962227e-02 -4.20557684e-02
   6.10512820e-02 -1.43605053e-02 -3.38836709e-03  3.54195845e-01
  -2.28451371e-02  1.57287407e-01 -1.75408027e-02  4.12487458e-02]
 [-8.56028698e-02 -7.87067999e-02 -2.68767044e-03  1.05252528e-02
  -3.10615005e-02  1.58878119e-02  8.39924240e-02  2.79036657e-02
   8.40551731e-03 -1.13825807e-01 -7.57952316e-02 -2.28451371e-02
   1.83017606e-01 -9.57386778e-03 -3.28961077e-02 -2.35661656e-02]
 [-1.54268159e-01  4.22005279e-02 -4.26210810e-02  7.47622852e-03
  -7.53456283e-02 -1.91601853e-02 -1.21590634e-02  4.00113245e-02
   1.07545489e-02  1.03525418e-01 -4.78962089e-02  1.57287407e-01
  -9.57386778e-03  1.38365005e-01  2.87665868e-02 -3.85542355e-03]
 [-4.72568542e-03  3.31667533e-02 -1.87647553e-02  6.99673784e-02
   3.80455937e-02 -6.39928820e-03  2.79901417e-02  2.25522235e-03
   1.80616644e-02  2.83620174e-02 -2.78042911e-02 -1.75408027e-02
  -3.28961077e-02  2.87665868e-02  9.10441260e-02 -2.66561455e-02]
 [ 2.97263996e-02  1.94539082e-02  7.61803231e-02 -1.72955169e-02
   2.31933417e-02  5.50329717e-02  1.39571204e-02  3.40051782e-02
   1.35292840e-02 -1.75226862e-03 -9.59508563e-03  4.12487458e-02
  -2.35661656e-02 -3.85542355e-03 -2.66561455e-02  4.05440847e-02]]